# import the nescessary modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor #ensemble---group
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = pd.read_excel("C:/Users/ELCOT/Downloads/Flight .xlsx")

In [3]:
df.head()

,Person,Item_Identifier,Length,Breadth,PaperQuality,Angle,Foldings,Windspeed,Dominanthand,Distance
0,Kevin,Som,29.0,20.0,A,U,8.0,On,L,5.0
1,Kevin,kevin,23.0,16.0,B,U,8.0,On,L,5.0
2,Kevin,RR,NaN,18.0,A,L,10.0,On,R,6.0
3,Priya,Nithya,27.0,20.0,A,U,10.0,On,R,4.0
4,Priya,Priya,29.5,17.5,B,U,5.0,Off,R,8.0


In [4]:
df.isnull().sum()

Person              0
Item_Identifier     0
Length             17
Breadth             0
PaperQuality        8
Angle               4
Foldings            0
Windspeed           0
Dominanthand        0
Distance            0
dtype: int64

In [5]:
df.Length.fillna(df.Length.mean(),inplace=True)

In [6]:
q3 = df["Length"].quantile(0.75)  
q1 = df["Length"].quantile(0.25)
IQR = q3 - q1
lower_limit = q1 - (IQR * 1.5)
upper_limit = q3 + (IQR * 1.5)

In [7]:
df['Length'] = np.where(df['Length'] >= upper_limit,upper_limit,np.where(df['Length'] <= lower_limit,lower_limit,df['Length']))

In [8]:
for j in df[df['PaperQuality'].isnull()].index.tolist():
    df.PaperQuality.iloc[j] =  df.PaperQuality[df['Item_Identifier']==df['Item_Identifier'].iloc[j]].mode().max()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
for j in df[df['Angle'].isnull()].index.tolist():
    df.Angle.iloc[j] =  df.Angle[df['Person']==df['Person'].iloc[j]].mode().max()

In [10]:
df.columns = df.columns.str.replace(' ','')

In [11]:
df.insert(9,'isDominant','Yes')

In [12]:
for i in range(88):
    if df.Dominanthand[df['Person']==df.Person.iloc[i]].mode().max() != df.Dominanthand.iloc[i]:
        df.isDominant.iloc[i] = 'No'

In [13]:
df.drop(['Person','Item_Identifier'],axis=1,inplace=True)

In [14]:
df.head(7)

,Length,Breadth,PaperQuality,Angle,Foldings,Windspeed,Dominanthand,isDominant,Distance
0,29.000000,20.0,A,U,8.0,On,L,Yes,5.0
1,23.000000,16.0,B,U,8.0,On,L,Yes,5.0
2,26.708451,18.0,A,L,10.0,On,R,No,6.0
3,27.000000,20.0,A,U,10.0,On,R,Yes,4.0
4,29.500000,17.5,B,U,5.0,Off,R,Yes,8.0
5,26.708451,18.5,B,L,5.0,Off,R,Yes,6.0
6,30.000000,19.0,B,U,8.0,Off,R,Yes,7.0


In [15]:
df['Distance'].value_counts()

6.0     20
5.0     18
7.0     17
8.0      8
3.0      7
4.0      6
9.0      6
11.0     2
1.0      2
10.0     2
Name: Distance, dtype: int64

In [16]:
df.loc[ df["Distance"] < 6, "Distance"] = 0
df.loc[df["Distance"] != 0, "Distance"] = 1

### for val <=6 is represented as 0
### for val >=5 is represented as 1

In [17]:
df['Distance'].value_counts()

1.0    55
0.0    33
Name: Distance, dtype: int64

In [18]:
df.head(8)

,Length,Breadth,PaperQuality,Angle,Foldings,Windspeed,Dominanthand,isDominant,Distance
0,29.000000,20.0,A,U,8.0,On,L,Yes,0.0
1,23.000000,16.0,B,U,8.0,On,L,Yes,0.0
2,26.708451,18.0,A,L,10.0,On,R,No,1.0
3,27.000000,20.0,A,U,10.0,On,R,Yes,0.0
4,29.500000,17.5,B,U,5.0,Off,R,Yes,1.0
5,26.708451,18.5,B,L,5.0,Off,R,Yes,1.0
6,30.000000,19.0,B,U,8.0,Off,R,Yes,1.0
7,25.000000,17.0,B,U,8.0,On,R,Yes,1.0


In [19]:
df.columns

Index(['Length', 'Breadth', 'PaperQuality', 'Angle', 'Foldings', 'Windspeed',
       'Dominanthand', 'isDominant', 'Distance'],
      dtype='object')

In [20]:
dfd = pd.get_dummies(df)

In [21]:
dfd.head()

,Length,Breadth,Foldings,Distance,PaperQuality_A,PaperQuality_B,Angle_L,Angle_U,Windspeed_Off,Windspeed_On,Dominanthand_L,Dominanthand_R,isDominant_No,isDominant_Yes
0,29.000000,20.0,8.0,0.0,1,0,0,1,0,1,1,0,0,1
1,23.000000,16.0,8.0,0.0,0,1,0,1,0,1,1,0,0,1
2,26.708451,18.0,10.0,1.0,1,0,1,0,0,1,0,1,1,0
3,27.000000,20.0,10.0,0.0,1,0,0,1,0,1,0,1,0,1
4,29.500000,17.5,5.0,1.0,0,1,0,1,1,0,0,1,0,1


In [22]:
df.isnull().sum()

Length          0
Breadth         0
PaperQuality    0
Angle           0
Foldings        0
Windspeed       0
Dominanthand    0
isDominant      0
Distance        0
dtype: int64

In [23]:
scaler = MinMaxScaler()
scaler.fit(dfd)

MinMaxScaler()

In [24]:
scalerData = scaler.transform(dfd)
scalerData = pd.DataFrame(scalerData)

In [25]:
scalerData.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.467809,0.555556,0.627273,0.625000,0.409091,0.590909,0.250000,0.750000,0.500000,0.500000,0.204545,0.795455,0.136364,0.863636
std,0.198449,0.297571,0.347314,0.486897,0.494484,0.494484,0.435494,0.435494,0.502865,0.502865,0.405681,0.405681,0.345141,0.345141
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.375000,0.333333,0.600000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
50%,0.485423,0.555556,0.600000,1.000000,0.000000,1.000000,0.000000,1.000000,0.500000,0.500000,0.000000,1.000000,0.000000,1.000000
75%,0.625000,0.888889,1.000000,1.000000,1.000000,1.000000,0.250000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
x = dfd.drop(['Distance'],axis=1)

In [27]:
y = dfd['Distance']

In [28]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.2,random_state=42)

## random forest regression

In [29]:
regressor=RandomForestRegressor(random_state=42)

In [30]:
regressor.fit(train_x,train_y)

RandomForestRegressor(random_state=42)

In [31]:
regressor.predict(test_x)

array([0.64603571, 0.99107143, 0.12583333, 0.3479246 , 0.82005952,
       0.64603571, 0.662     , 0.91166667, 0.73249603, 0.57540476,
       0.85271734, 0.6371443 , 0.23055952, 0.65427778, 0.91166667,
       0.57042044, 0.75544048, 0.99107143])

In [32]:
regressor.predict(train_x)

array([0.75544048, 0.57042044, 0.75544048, 0.6371443 , 0.99314286,
       0.1202381 , 0.23055952, 0.23055952, 0.91166667, 0.51466667,
       0.23055952, 0.87071429, 0.9251057 , 0.99107143, 0.40835714,
       0.57042044, 0.85177778, 0.57042044, 0.99314286, 0.73249603,
       0.85271734, 0.85271734, 0.98190476, 0.20020238, 0.99107143,
       0.75544048, 0.20020238, 0.51466667, 0.20020238, 0.09157143,
       0.98190476, 0.85271734, 0.98190476, 0.65427778, 0.98190476,
       0.6371443 , 0.64603571, 0.6371443 , 0.85271734, 0.36938095,
       0.57042044, 0.36938095, 0.73249603, 0.1325    , 0.85271734,
       0.91166667, 0.9251057 , 0.73249603, 0.75544048, 0.99107143,
       0.64603571, 0.73249603, 0.31965079, 0.85271734, 0.23055952,
       0.36938095, 0.57042044, 0.65427778, 0.6371443 , 0.6371443 ,
       0.31965079, 0.64603571, 0.31965079, 0.6371443 , 0.40835714,
       0.57042044, 0.20020238, 0.99314286, 0.65427778, 0.57042044])

In [33]:
regressor.score(test_x,regressor.predict(test_x))

1.0

## GradientBoosting Classifier

In [34]:
mgc=GradientBoostingClassifier(n_estimators=100,max_depth=2,random_state=42)

In [35]:
mgc.fit(train_x,train_y)

GradientBoostingClassifier(max_depth=2, random_state=42)

In [36]:
mean_squared_error(test_y,mgc.predict(test_x),squared=False)

0.7817359599705717



### Logistic Regression

In [37]:
logreg=LogisticRegression()

In [38]:
logreg.fit(train_x,train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [40]:
mean_squared_error(test_y,logreg.predict(test_x),squared=False)

0.7817359599705717

### Decision tree class

In [41]:
modeldtc = tree.DecisionTreeClassifier(random_state=42)
modeldtc.fit(train_x,train_y)

DecisionTreeClassifier(random_state=42)

In [42]:
mean_squared_error(test_y,modeldtc.predict(test_x),squared=False)

0.7071067811865476

### predictions

In [47]:
logreg.predict(test_x)

array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
       1.])

In [48]:
accuracy_score(test_y, logreg.predict(test_x))

0.3888888888888889

In [49]:
test_y

76    1.0
0     0.0
26    1.0
22    1.0
12    0.0
67    1.0
10    0.0
18    1.0
4     1.0
68    0.0
85    0.0
65    0.0
53    1.0
80    0.0
84    0.0
64    1.0
33    1.0
79    1.0
Name: Distance, dtype: float64

In [51]:
mean_squared_error(test_y, logreg.predict(test_x), squared=False)

0.7817359599705717

## AdaBoosting Classifier

In [52]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics

In [53]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model = abc.fit(train_x,train_y)

In [54]:
pred = model.predict(test_x)

In [55]:
mean_squared_error(test_y, model.predict(test_x), squared=False)

0.7453559924999299